# 1

In [20]:
!pip install gffpandas

     |████████████████████████████████| 178 kB 19.0 MB/s 
  Created wheel for gffpandas: filename=gffpandas-1.2.0-py2.py3-none-any.whl size=6263 sha256=dd328a31c4f7c36c247a73bfb919dce5a6257d6778aa9301dfad8d7e325878c0
  Stored in directory: /root/.cache/pip/wheels/a9/d0/2a/cb5ebf14451176d87e738547d58cb2ec39ebc46b19649133cc
Successfully built gffpandas


In [28]:
import gffpandas.gffpandas as gffpd
import numpy as np
import pandas as pd

1. считываем **gff** файл, фильтруем его, отбирая только данные имеющие type  = **gene**, переводим его в csv чтобы потом работать с ним как с датафреймом с помощью **pandas**
2. создаем датафрейм **genes** из файла csv
3. создаем новую колонку **length**, которая будет содержать длины всех генов

In [91]:
ann = gffpd.read_gff3('annotation.gff')
ann.filter_feature_of_type(['gene']).to_csv('genes.csv')
genes = pd.read_csv('genes.csv', delimiter=',')
genes['length'] = genes['end'] - genes['start'] + 1
genes.head()

,seq_id,source,type,start,end,score,strand,phase,attributes,length
0,NC_045512.2,RefSeq,gene,266,21555,.,+,.,ID=gene-GU280_gp01;Dbxref=GeneID:43740578;Name...,21290
1,NC_045512.2,RefSeq,gene,21563,25384,.,+,.,ID=gene-GU280_gp02;Dbxref=GeneID:43740568;Name...,3822
2,NC_045512.2,RefSeq,gene,25393,26220,.,+,.,ID=gene-GU280_gp03;Dbxref=GeneID:43740569;Name...,828
3,NC_045512.2,RefSeq,gene,26245,26472,.,+,.,ID=gene-GU280_gp04;Dbxref=GeneID:43740570;Name...,228
4,NC_045512.2,RefSeq,gene,26523,27191,.,+,.,ID=gene-GU280_gp05;Dbxref=GeneID:43740571;Name...,669


тут можно закончить, но мне не нравится колонка attributes, хотелось бы иметь нормальный датафрейм, а не это

Посмотрим как выглядит **attributes**

In [81]:
print(genes['attributes'][0])

ID=gene-GU280_gp01;Dbxref=GeneID:43740578;Name=ORF1ab;gbkey=Gene;gene=ORF1ab;gene_biotype=protein_coding;locus_tag=GU280_gp01


Понятно, что в исходном csv файле просто было два разделителя (; и ,) и можно было бы просто заменить один на другой (; на ,) и проблема была бы решена, но я не знаю как это сделать, поэтому пришлось идти окольными путями 

Вобщем я решил просто сделать из колонки **genes['attributes']** датафрейм и затем присоединить его к исходному

In [92]:
att = pd.DataFrame([[j[j.index('=') + 1:] for j in i.split(';')] for i in genes['attributes'].tolist()], 
                   columns = ['ID', 'GeneID', 'Name', 'gbkey', 'gene', 'gene_biotype', 'locus_tag', 'None'])
att

,ID,GeneID,Name,gbkey,gene,gene_biotype,locus_tag,None
0,gene-GU280_gp01,GeneID:43740578,ORF1ab,Gene,ORF1ab,protein_coding,GU280_gp01,None
1,gene-GU280_gp02,GeneID:43740568,S,Gene,S,protein_coding,spike glycoprotein,GU280_gp02
2,gene-GU280_gp03,GeneID:43740569,ORF3a,Gene,ORF3a,protein_coding,GU280_gp03,None
3,gene-GU280_gp04,GeneID:43740570,E,Gene,E,protein_coding,GU280_gp04,None
4,gene-GU280_gp05,GeneID:43740571,M,Gene,M,protein_coding,GU280_gp05,None
5,gene-GU280_gp06,GeneID:43740572,ORF6,Gene,ORF6,protein_coding,GU280_gp06,None
6,gene-GU280_gp07,GeneID:43740573,ORF7a,Gene,ORF7a,protein_coding,GU280_gp07,None
7,gene-GU280_gp08,GeneID:43740574,ORF7b,Gene,ORF7b,protein_coding,GU280_gp08,None
8,gene-GU280_gp09,GeneID:43740577,ORF8,Gene,ORF8,protein_coding,GU280_gp09,None
9,gene-GU280_gp10,GeneID:43740575,N,Gene,N,protein_coding,GU280_gp10,None


Раз мы получили из колонки датафрейм, то исходная колонка нам уже не нужна, можем удалить ее

In [93]:
genes = genes.drop(['attributes'], axis = 1)

Наконец соединим два датафрейма в один

In [94]:
pd.concat([genes, att], axis = 1)

,seq_id,source,type,start,end,score,strand,phase,length,ID,GeneID,Name,gbkey,gene,gene_biotype,locus_tag,None
0,NC_045512.2,RefSeq,gene,266,21555,.,+,.,21290,gene-GU280_gp01,GeneID:43740578,ORF1ab,Gene,ORF1ab,protein_coding,GU280_gp01,None
1,NC_045512.2,RefSeq,gene,21563,25384,.,+,.,3822,gene-GU280_gp02,GeneID:43740568,S,Gene,S,protein_coding,spike glycoprotein,GU280_gp02
2,NC_045512.2,RefSeq,gene,25393,26220,.,+,.,828,gene-GU280_gp03,GeneID:43740569,ORF3a,Gene,ORF3a,protein_coding,GU280_gp03,None
3,NC_045512.2,RefSeq,gene,26245,26472,.,+,.,228,gene-GU280_gp04,GeneID:43740570,E,Gene,E,protein_coding,GU280_gp04,None
4,NC_045512.2,RefSeq,gene,26523,27191,.,+,.,669,gene-GU280_gp05,GeneID:43740571,M,Gene,M,protein_coding,GU280_gp05,None
5,NC_045512.2,RefSeq,gene,27202,27387,.,+,.,186,gene-GU280_gp06,GeneID:43740572,ORF6,Gene,ORF6,protein_coding,GU280_gp06,None
6,NC_045512.2,RefSeq,gene,27394,27759,.,+,.,366,gene-GU280_gp07,GeneID:43740573,ORF7a,Gene,ORF7a,protein_coding,GU280_gp07,None
7,NC_045512.2,RefSeq,gene,27756,27887,.,+,.,132,gene-GU280_gp08,GeneID:43740574,ORF7b,Gene,ORF7b,protein_coding,GU280_gp08,None
8,NC_045512.2,RefSeq,gene,27894,28259,.,+,.,366,gene-GU280_gp09,GeneID:43740577,ORF8,Gene,ORF8,protein_coding,GU280_gp09,None
9,NC_045512.2,RefSeq,gene,28274,29533,.,+,.,1260,gene-GU280_gp10,GeneID:43740575,N,Gene,N,protein_coding,GU280_gp10,None


# 2

тут не было ограничения на сложность или память, поэтому реализовал самый тупой алгоритм с созданием массива сдвигов - **a**, который отсортировал, и затем создал требуемый суффиксный массив, в котором просто прошелся по отсортированному массиву сдвигов и посмотрел на какой позиции в неотсортированном массиве сдвигов стояли те или иные слова

In [18]:
x = 'ATATATTAG$'
a = [x[i:] + x[:i] for i in range(len(x))]
s = [a.index(i) + 1 for i in sorted(a)]
print(s)

[10, 8, 1, 3, 5, 9, 7, 2, 4, 6]


# 3


Несколько алгоритмов подходят сразу под создание bwt

Первый алгоритм это какая то магия, второй выглядит как "создай сдвиг и затем скопируй у него последнюю букву", а третий это тот исходный алгоритм который описывался в лекции

In [87]:
x = 'ATATATTAG$'
s = [10, 8, 1, 3, 5, 9, 7, 2, 4, 6]
ans = 'GT$TTATAAA'

bwt = ''
for i in s:
  bwt += x[abs(1 - i) - 1]
print(bwt)

bwt_1 = ''
for i in s:
  new = x[i - 1:] + x[:i - 1]
  bwt_1 += new[-1]
print(bwt_1)

x = 'ATATATTAG'
bwt_2 = ''
for i in s:
  if i == 1:
    bwt_2 += '$'
  else:
    bwt_2 += x[i - 2]
print(bwt_2)

GT$TTATAAA
GT$TTATAAA
GT$TTATAAA


Функция C

По сложности алгоритм линеен - $O(k\cdot n) = O(n)$

In [63]:
C = [['$', 0], ['A', 0], ['C', 0], ['G', 0], ['T', 0]]
x = 'ATATATTAG$'

for i in range(len(C) - 1):
  for j in x:
    if j == C[i][0]:
      C[i + 1][1] += 1
  C[i + 1][1] += C[i][1]

C = {i[0]: i[1] for i in C}
print(C)

{'$': 0, 'A': 1, 'C': 5, 'G': 5, 'T': 6}


# 4

перебираем все перестановки букв в bwt, для каждой перестановки (каждая перестановка является кандидатом на исходное слово) создаем суффиксный массив и находим bwt_new, если bwt_new и bwt совпадают, то данная перестановка является исходным словом

# 5

5 - минимальное число мутаций, при котором можно придумать случай, при котором нет чистых k-меров длины 20

(мутации на 20, 40, 60, 80 и 100 нуклеотиде)

При 4 мутациях всегда есть хотя бы один чистый 20-мер

Ответ: 4